In [13]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit = 80,
    return_messages = True,
) 

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name = "history"),
    ("human","{question}")
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory)|prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context({"input":question},{"output":result.content})
    print(result)

invoke_chain("My name is Nico.")
memory.load_memory_variables({})

content='Hello Nico! How can I assist you today?'


{'history': [HumanMessage(content='My name is Nico.'),
  AIMessage(content='Hello Nico! How can I assist you today?')]}

In [10]:
invoke_chain("I live in Suwon")


{'question': 'I live in Suwon'}
content="That's great, Nico! Suwon is a wonderful city with a rich history and modern amenities. Is there anything you'd like to know or discuss about living in Suwon? Feel free to ask any questions or share your experiences!"


In [11]:
invoke_chain("what is my name?")

{'question': 'what is my name?'}
content='Your name is Nico. How can I assist you further, Nico?'


In [12]:
memory.load_memory_variables({})

{'chat_history': [SystemMessage(content='Nico introduces themselves to the AI, mentioning they live in Suwon. The AI greets Nico and talks about the beauty of Suwon, known for its historical sites and vibrant culture. The AI invites Nico to ask any questions or share experiences about living in Suwon.'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Nico. How can I assist you further, Nico?')]}